---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx
import pandas as pd

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one():
    email = nx.read_edgelist('email_network.txt', data=[('time', int)], 
                         create_using=nx.MultiDiGraph())
    
    # Your Code Here
    
    return email
answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [3]:
def answer_two():
    email = answer_one()
    #print(email.degree())
    df = pd.DataFrame(email.edges(data=True), columns=['Sender', 'Recipient', 'time'])
    #print(df. head)
    senderlist = df.Sender.value_counts().index.tolist()
    recipientlist = df.Recipient.value_counts().index.tolist()
    employeelist = pd.Series(recipientlist + senderlist)
    employeelist.nunique()
        
    return (employeelist.nunique(), df.Sender.count())
answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [4]:
def answer_three():
        
    email = answer_one()
    part1 = nx.is_strongly_connected(email)
    part2 = nx.is_weakly_connected(email)
    
    
    return (part1, part2)
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [5]:
def answer_four():
    email = answer_one()
    
    ans = len(sorted(nx.weakly_connected_components(email))[0])
    # Your Code Here
    
    return ans
answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [6]:
def answer_five():
    email = answer_one()
    stronglist = sorted(nx.strongly_connected_components(email))
    
    list_lengths = [len(i) for i in stronglist]
    ans = max(list_lengths)

    return ans
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [7]:
def answer_six():
    email = answer_one()
    G_sc = max(nx.strongly_connected_component_subgraphs(email), key=len)
    
    
    return G_sc 
answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [8]:
def answer_seven():
    G_sc = answer_six()
    ans = nx.average_shortest_path_length(G_sc)
    
    return ans
answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [9]:
def answer_eight():
    
    G_sc = answer_six()
    ans = nx.diameter(G_sc)
    
    return ans
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [10]:
def answer_nine():
    G_sc = answer_six()
    G_sc_dia = answer_eight()
    
    ecc_dict = nx.eccentricity(G_sc)
    
    #print(ecc_dict)
    
    nodes = []
    
    for k,v in ecc_dict.items():
        if v == G_sc_dia:
            nodes.append(k)
    node_set = set(nodes)    
    return node_set
answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [11]:
def answer_ten():
    G_sc = answer_six()
    G_sc_rad = nx.radius(G_sc)
    
    ecc_dict = nx.eccentricity(G_sc)
        
    nodes = []
    
    for k,v in ecc_dict.items():
        if v == G_sc_rad:
            nodes.append(k)
    node_set = set(nodes)   

    
    print(G_sc_rad)
    
    return node_set
answer_ten()

1


{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [12]:
def answer_eleven():
    from collections import Counter
    G_sc = answer_six()
    
    G_sc_spl = nx.shortest_path_length(G_sc)
    G_sc_dia = nx.diameter(G_sc)
    
    #print(G_sc_spl)
    nodes = []
    
    for k,v in G_sc_spl.items():
        for key, value in v.items():
            if value == G_sc_dia:
                nodes.append(k)
    
    count = Counter(nodes).most_common(1)   
    
    return count[0]
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [13]:
def answer_twelve():
    G_sc = answer_six()
    a11_node = answer_eleven()[0]
    center_nodes = nx.center(G_sc)
    
    minnodecut = nx.minimum_node_cut(G_sc, t=a11_node, s=center_nodes[0])
    
    return len(minnodecut)
answer_twelve()

5

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [16]:
def answer_thirteen():
    G_Sc = answer_six()
    G_Sc_undirected = G_Sc.to_undirected()
    G_un = nx.Graph(G_Sc_undirected)
            
    return G_un
answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [18]:
def answer_fourteen():
    G_un = answer_thirteen()
    avg_clustering = nx.average_clustering(G_un)
    transitivity = nx.transitivity(G_un)
    
    return (transitivity, avg_clustering)
answer_fourteen()

(0.570111160700385, 0.697527243723142)